In [20]:
GENE_SELECTED = "DIS3L2"


In [21]:
import os
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import concurrent.futures
sns.set(style="whitegrid")

In [22]:
map_file = '../data/22Q2//Achilles_guide_map.csv'
data_file = '../data/22Q2/Achilles_gene_effect.csv'
map = pd.read_csv(map_file)
achilles = pd.read_csv(data_file)

Let's see the files

In [23]:
map.head(2)

,sgrna,genome_alignment,gene,n_alignments
0,AAAAAAATCCAGCAATGCAG,chr10_110964620_+,SHOC2 (8036),1
1,AAAAAACCCGTAGATAGCCT,chr12_95003615_+,NDUFA12 (55967),1


In [24]:
map.shape

(74336, 4)

In [25]:
achilles.head(2)

,DepMap_ID,A1BG (1),A1CF (29974),A2M (2),A2ML1 (144568),A3GALT2 (127550),A4GALT (53947),A4GNT (51146),AAAS (8086),AACS (65985),...,ZWILCH (55055),ZWINT (11130),ZXDA (7789),ZXDB (158586),ZXDC (79364),ZYG11A (440590),ZYG11B (79699),ZYX (7791),ZZEF1 (23140),ZZZ3 (26009)
0,ACH-000004,0.101156,0.019349,-0.218404,-0.010704,0.146463,-0.142147,0.148280,-0.329886,0.158813,...,0.027789,-0.222223,0.028758,-0.042301,0.118614,-0.058945,-0.253932,0.19573,0.151277,-0.023335
1,ACH-000005,-0.028238,0.042131,0.004066,0.108121,0.111987,-0.137859,0.121038,-0.359282,-0.045626,...,0.080696,-0.272542,-0.251188,-0.133216,0.015223,-0.034227,-0.248819,0.09712,-0.025372,-0.091575


In [26]:
achilles.shape

(957, 18018)

#### Some preprocessing

In [27]:
achilles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957 entries, 0 to 956
Columns: 18018 entries, DepMap_ID to ZZZ3 (26009)
dtypes: float64(18017), object(1)
memory usage: 131.6+ MB


In [28]:
achilles = achilles.dropna()

In [29]:

achilles.columns = achilles.columns.str.replace(" ", "_").str.replace("[()]", "", regex=True)

In [30]:
achilles.head(2)

,DepMap_ID,A1BG_1,A1CF_29974,A2M_2,A2ML1_144568,A3GALT2_127550,A4GALT_53947,A4GNT_51146,AAAS_8086,AACS_65985,...,ZWILCH_55055,ZWINT_11130,ZXDA_7789,ZXDB_158586,ZXDC_79364,ZYG11A_440590,ZYG11B_79699,ZYX_7791,ZZEF1_23140,ZZZ3_26009
0,ACH-000004,0.101156,0.019349,-0.218404,-0.010704,0.146463,-0.142147,0.148280,-0.329886,0.158813,...,0.027789,-0.222223,0.028758,-0.042301,0.118614,-0.058945,-0.253932,0.19573,0.151277,-0.023335
1,ACH-000005,-0.028238,0.042131,0.004066,0.108121,0.111987,-0.137859,0.121038,-0.359282,-0.045626,...,0.080696,-0.272542,-0.251188,-0.133216,0.015223,-0.034227,-0.248819,0.09712,-0.025372,-0.091575


In [31]:
achilles.columns = achilles.columns.str.replace(" ", "_").str.replace("[()]", "", regex=True)
achilles_clean= pd.melt(achilles, id_vars=["DepMap_ID"], var_name="genes", value_name="value_gene_selected")
achilles_clean.head()

,DepMap_ID,genes,value_gene_selected
0,ACH-000004,A1BG_1,0.101156
1,ACH-000005,A1BG_1,-0.028238
2,ACH-000007,A1BG_1,0.017333
3,ACH-000009,A1BG_1,0.010019
4,ACH-000011,A1BG_1,0.148716


#### Maybe store the number in other column? 

In [32]:
genes = pd.Series(achilles_clean.genes.unique())
genes[genes.str.startswith(GENE_SELECTED)]


4079    DIS3L2_129563
dtype: object

In [33]:
achilles_clean[['gene', 'gene_id']] = achilles_clean['genes'].str.split('_', expand=True)


In [34]:
achilles_clean.drop(columns=['genes'], inplace = True)
achilles_clean.head()

,DepMap_ID,value_gene_selected,gene,gene_id
0,ACH-000004,0.101156,A1BG,1
1,ACH-000005,-0.028238,A1BG,1
2,ACH-000007,0.017333,A1BG,1
3,ACH-000009,0.010019,A1BG,1
4,ACH-000011,0.148716,A1BG,1


In [35]:
achilles_clean.shape

(17152184, 4)

In [36]:
map[['chromosome', 'avg_pos','_']] = map['genome_alignment'].str.split('_', expand=True)
map[['gene', 'gene_id']] = map['gene'].str.split(' ', expand=True)
map.gene_id = map.gene_id.str.replace("[()]", "", regex=True)
map.head()

,sgrna,genome_alignment,gene,n_alignments,chromosome,avg_pos,_,gene_id
0,AAAAAAATCCAGCAATGCAG,chr10_110964620_+,SHOC2,1,chr10,110964620,+,8036
1,AAAAAACCCGTAGATAGCCT,chr12_95003615_+,NDUFA12,1,chr12,95003615,+,55967
2,AAAAAAGAAGAAAAAACCAG,chr4_75970356_-,SDAD1,1,chr4,75970356,-,55153
3,AAAAAAGCTCAAGAAGGAGG,chr2_33588446_-,FAM98A,1,chr2,33588446,-,25940
4,AAAAAAGGCTGTAAAAGCGT,chr19_19891600_+,ZNF253,1,chr19,19891600,+,56242


#### Let's compute the correlations

In [37]:
achilles_clean.head()

,DepMap_ID,value_gene_selected,gene,gene_id
0,ACH-000004,0.101156,A1BG,1
1,ACH-000005,-0.028238,A1BG,1
2,ACH-000007,0.017333,A1BG,1
3,ACH-000009,0.010019,A1BG,1
4,ACH-000011,0.148716,A1BG,1


In [38]:
data = achilles_clean.copy()
selected_gene_data = data[data['gene'] == GENE_SELECTED]['value_gene_selected']
selected_gene_data.head()

3883208   -0.096076
3883209    0.258862
3883210   -0.076563
3883211    0.099047
3883212    0.035837
Name: value_gene_selected, dtype: float64

In [21]:
data = achilles_clean.copy()
selected_gene_data = data[data['gene'] == GENE_SELECTED]['value_gene_selected']

# Create an empty dictionary to store correlations
correlations = []

gene_count = 0
def calculate_correlation(selected_gene, other_gene_data):
    correlation, pvalue = pearsonr(selected_gene, other_gene_data)
    return correlation, pvalue, -np.log10(pvalue)


# Get unique genes in the dataset
unique_genes = data['gene'].unique()

correlations = []

# correlations for a single gene
def calculate_gene_correlations(other_gene):
    other_gene_data = data[data['gene'] == other_gene]['value_gene_selected']
    correlation, pvalue, logpvalue = calculate_correlation(selected_gene_data, other_gene_data)
    return {'SELECTED_GENE': GENE_SELECTED, 'EVALUATED_GENE': other_gene, 'CORRELATION': correlation, 'PVALUE': pvalue, 'LOGPVALUE': logpvalue}

# concurrent.futures to parallelize the computation
with concurrent.futures.ProcessPoolExecutor() as executor:
    for gene_count, result in enumerate(executor.map(calculate_gene_correlations, unique_genes), 1):
        correlations.append(result)
        
        # Print progress 
        if gene_count % 1000 == 0:
            print(f"Computed correlations for {gene_count} / {len(unique_genes)} genes.")


results = pd.DataFrame(correlations)
FILENAME = "../output/correlations/correlations_" + GENE_SELECTED + "_by_chr.csv"

# Save the results to a CSV file
results.to_csv(FILENAME, index=False)

# Display the first few rows of the results
results.head()

Computed correlations for 1000 / 18119 genes.
Computed correlations for 2000 / 18119 genes.
Computed correlations for 3000 / 18119 genes.
Computed correlations for 4000 / 18119 genes.


/scratch/usertmp/ipykernel_3839478/2887426482.py:10: RuntimeWarning: divide by zero encountered in log10
  return correlation, pvalue, -np.log10(pvalue)


Computed correlations for 5000 / 18119 genes.
Computed correlations for 6000 / 18119 genes.
Computed correlations for 7000 / 18119 genes.
Computed correlations for 8000 / 18119 genes.
Computed correlations for 9000 / 18119 genes.
Computed correlations for 10000 / 18119 genes.
Computed correlations for 11000 / 18119 genes.
Computed correlations for 12000 / 18119 genes.
Computed correlations for 13000 / 18119 genes.
Computed correlations for 14000 / 18119 genes.
Computed correlations for 15000 / 18119 genes.
Computed correlations for 16000 / 18119 genes.
Computed correlations for 17000 / 18119 genes.
Computed correlations for 18000 / 18119 genes.


,SELECTED_GENE,EVALUATED_GENE,CORRELATION,PVALUE,LOGPVALUE
0,DIS3L2,A1BG,0.018735,0.601819,0.220534
1,DIS3L2,A1CF,-0.017360,0.628761,0.201514
2,DIS3L2,A2M,-0.023249,0.517303,0.286255
3,DIS3L2,A2ML1,-0.058463,0.103218,0.986244
4,DIS3L2,A3GALT2,0.019349,0.589967,0.229173
